In [12]:
import findspark
findspark.init('/home/gerardo-rodriguez/spark-4.0.0-bin-hadoop3')

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
import time

In [14]:
spark = SparkSession.builder.appName('twitter_app').getOrCreate()

In [15]:
socket_stream = spark.readStream.format('socket').option('host', 'localhost').option('port', 5555).load()\
.withColumn('timestamp', F.current_timestamp())

25/08/22 12:59:39 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [16]:
hastag_counts = socket_stream.select(
    F.explode(F.split(F.col('value'), " ")).alias('word')
).filter(
        F.lower(F.col('word')).startswith('#')
).groupBy(
        F.lower(F.col('word')).alias('tag')
).count()

query = hastag_counts.writeStream.outputMode('complete').format('memory').queryName('tweets').start()

try:
    while True:
        print("\n=== Top 10 Hashtags ===")
        spark.sql("SELECT * FROM tweets ORDER BY count DESC LIMIT 10").show(truncate=False)
        time.sleep(10)
except KeyboardInterrupt:
    print("Stopping streaming...")
    query.stop()


25/08/22 12:59:40 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-c7475bf4-4c81-4035-ae86-201078c288d8. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/08/22 12:59:40 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.



=== Top 10 Hashtags ===
+---+-----+
|tag|count|
+---+-----+
+---+-----+




=== Top 10 Hashtags ===
+---+-----+
|tag|count|
+---+-----+
+---+-----+

Stopping streaming...


25/08/22 12:59:52 WARN DAGScheduler: Failed to cancel job group 858152e3-95b2-4ff1-aad1-3b5d220acdd6. Cannot find active jobs for it.
25/08/22 12:59:52 WARN DAGScheduler: Failed to cancel job group 858152e3-95b2-4ff1-aad1-3b5d220acdd6. Cannot find active jobs for it.
